<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW2/blob/main/BigData_HW2_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import pandas as pd
import numpy as np

# Dataset

In [2]:
!gdown 1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
!unzip /content/Bigdata_hw2_datasets.zip

Downloading...
From: https://drive.google.com/uc?id=1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 124MB/s]
Archive:  /content/Bigdata_hw2_datasets.zip
replace Bigdata_hw2_datasets/q1/stream_data_dgim.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
replace Bigdata_hw2_datasets/q2/games.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [3]:
games_dataset = pd.read_csv("/content/Bigdata_hw2_datasets/q2/games.csv")

In [4]:
games_dataset = games_dataset[games_dataset["game_id"].notna()]

In [61]:
ratings_dataset = pd.read_csv("/content/Bigdata_hw2_datasets/q2/ratings.csv")

In [62]:
ratings_dataset = ratings_dataset.drop_duplicates(["user_id", "game_id"], keep="last")

# Recommendation Functions

In [66]:
def rated_item_list(user_id):
  return ratings_dataset[ratings_dataset["user_id"]==user_id]["game_id"].values

In [10]:
# def rater_user_list(item_id):
#   return ratings_dataset[ratings_dataset["game_id"]==item_id]["user_id"].values

In [11]:
# def mean_rate_user(user_id):
#   return ratings_dataset[ratings_dataset["user_id"]==user_id]["rating"].mean()

In [12]:
# def mean_rate_item(item_id):
#   return ratings_dataset[ratings_dataset["game_id"]==item_id]["rating"].mean()

In [13]:
# def rate(user_id, item_id):
#   return ratings_dataset[(ratings_dataset["user_id"]==user_id) & (ratings_dataset["game_id"]==item_id)]["rating"].values[0]

In [20]:
# def similarity_users(user_id_1, user_id_2):
#   common_item_list = np.intersect1d(rated_item_list(user_id_1), rated_item_list(user_id_2))

#   numerator = 0
#   denominator_1 = 0
#   denominator_2 = 0
  
#   mean_rate_user_1 = mean_rate_user(user_id_1)
#   mean_rate_user_2 = mean_rate_user(user_id_2)

#   if len(common_item_list) == 0:
#     return None

#   for item_id in common_item_list:
#     numerator += (rate(user_id_1, item_id)-mean_rate_user_1) * (rate(user_id_2, item_id)-mean_rate_user_2)    

#     denominator_1 += (rate(user_id_1, item_id)-mean_rate_user_1) ** 2
#     denominator_2 += (rate(user_id_2, item_id)-mean_rate_user_2) ** 2
    
#   return numerator / ((denominator_1 ** 0.5) * (denominator_2 ** 0.5))

In [23]:
# def similarity_items(item_id_1, item_id_2):
#   common_user_list = np.intersect1d(rater_user_list(item_id_1), rater_user_list(item_id_2))

#   numerator = 0
#   denominator_1 = 0
#   denominator_2 = 0
  
#   mean_rate_item_1 = mean_rate_item(item_id_1)
#   mean_rate_item_2 = mean_rate_item(item_id_2)

#   if len(common_user_list) == 0:
#     return None

#   for user_id in common_user_list:
#     numerator += (rate(user_id, item_id_1)-mean_rate_item_1) * (rate(user_id, item_id_2)-mean_rate_item_2)    

#     denominator_1 += (rate(user_id, item_id_1)-mean_rate_item_1) ** 2
#     denominator_2 += (rate(user_id, item_id_2)-mean_rate_item_2) ** 2
    
#   return numerator / ((denominator_1 ** 0.5) * (denominator_2 ** 0.5))

In [31]:
# ratings_dataset[ratings_dataset["user_id"] == 314]["rating"].values

array([5, 3, 4, 5, 4, 4, 4, 3, 3, 4, 5, 4, 4, 3, 4, 3, 4, 5, 2, 2, 2, 1,
       3, 4, 4, 4, 3, 4, 4, 3, 3, 4, 5, 4, 3, 4, 4, 3, 4, 3, 4, 3, 4, 4,
       5, 4, 4, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 5, 4, 2, 4, 5, 4, 3, 4, 4,
       5, 4, 5, 3, 4, 4, 3, 4, 3, 3, 2, 3, 4, 4, 4, 2, 4, 4, 3, 3, 4, 3,
       4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3, 4, 4, 4,
       4, 4, 4, 3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 3, 4, 3, 4, 4, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 4, 2, 4, 4, 3, 3, 3, 3, 3, 5, 4, 3, 5, 3,
       5, 3, 4, 4, 4, 3, 3, 4, 4, 3, 4, 4, 2, 4, 3, 4, 4, 3, 5, 2, 4, 4,
       3, 4, 4, 3, 3])

In [71]:
def cosine_similarity(user_x, user_y):
  if user_x == user_y:
    return 1

  user_x_rates = ratings_dataset[ratings_dataset["user_id"]==user_x]["rating"].values
  user_y_rates = ratings_dataset[ratings_dataset["user_id"]==user_y]["rating"].values

  user_x_norm = sum([rate**2 for rate in user_x_rates]) ** 0.5 
  user_y_norm = sum([rate**2 for rate in user_y_rates]) ** 0.5 

  common_item_list = np.intersect1d(rated_item_list(user_x), rated_item_list(user_y))

  minimal_ratings_dataset = ratings_dataset[ratings_dataset["game_id"].isin(common_item_list) & ratings_dataset["user_id"].isin([user_x, user_y])]

  user_x_y_dot = 0
  for item_id in common_item_list:
    rate_1, rate_2 = minimal_ratings_dataset[minimal_ratings_dataset["game_id"]==item_id]["rating"].values
    user_x_y_dot += rate_1 * rate_2

  return user_x_y_dot / (user_x_norm * user_y_norm)

In [72]:
all_user_id_list = set(ratings_dataset["user_id"].values)

In [81]:
def most_similar_users(user, number):
  similarity_scores = []

  for i, user_id in enumerate(all_user_id_list):
    if i<50500: continue
    if i%2000==0: print(int(i*100/len(all_user_id_list)), "%")
    if user_id != user:
      score = cosine_similarity(user, user_id)

      similarity_scores.append({"score": score, "user-id": user_id})

  similarity_scores = sorted(similarity_scores, key=lambda x: -x["score"])
  return similarity_scores[:number]

In [82]:
output = most_similar_users(user=1, number=10)

50880 0.12677794907094558
51296 0.1696438883438936
51446 0.2075123999456483
52021 0.23816275411477048
97 %
52725 0.17367722907355632
52938 0.05983493367951047


In [83]:
output

[{'score': 0.23816275411477048, 'user-id': 52021},
 {'score': 0.2075123999456483, 'user-id': 51446},
 {'score': 0.17367722907355632, 'user-id': 52725},
 {'score': 0.1696438883438936, 'user-id': 51296},
 {'score': 0.12677794907094558, 'user-id': 50880},
 {'score': 0.05983493367951047, 'user-id': 52938},
 {'score': 0.0, 'user-id': 50521},
 {'score': 0.0, 'user-id': 50522},
 {'score': 0.0, 'user-id': 50523},
 {'score': 0.0, 'user-id': 50524}]

# Results